# **라이브러리 정의**

# **라이브러리 정의**

In [1]:
import os
import pandas as pd
from supabase import create_client, Client

# **사용자 정보 초기화**

In [2]:
# 엔드포인트 URL 및 API 키 설정
SUPABASE_URL = 'https://yhayrbotkkuuvoxzhqct.supabase.co'
SUPABASE_KEY = 'sb_secret_5rUltxbkuiB3wFcTyMs1qw_cJHFo3kf'

# 클라이언트 연결
try:
    supabase: Client = create_client(SUPABASE_URL, SUPABASE_KEY)
    print("Supabase 클라이언트 연결 완료.")
except Exception as e:
    print(f"Supabase 클라이언트 연결 실패: {e}")

def validation(response):
    if response.data:
        print("데이터베이스 적용 완료")
    else:
        print("코드를 재확인해주세요.")

Supabase 클라이언트 연결 완료.


# **기본 문법**
: 모든 쿼리 로직 정의 이후 최종적으로 결과물을 얻으려면 아래 메서드를 이어붙여야함
- .execute().data

## **테이블 및 컬럼 선택**

In [ ]:
# Query : select * from quiz

# 1. 테이블 선택 (from)
select_table = supabase.from_('quiz')

# 2. 컬럼 선택 (select)
select_table_column = select_table.select('*')

## **특정 값 필터 : 단수**

In [ ]:
# 3-1. 특정 값 필터 (단수)
third = select_table_column.eq('category', 'Basic')

df = pd.DataFrame(third.execute().data)
df.head(2)

## **특정 값 필터 : 복수 (AND)**

In [ ]:
# 3-2. 특정 값 필터 (복수 : AND)
third_and = select_table_column.eq('category', 'Basic').eq('sub_category', '배당금')

df = pd.DataFrame(third_and.execute().data)
df.head(2)

## **특정 값 필터 : 복수 (OR)**
- 'category.eq.Basic' : category 컬럼의 값이 'Basic'
- 'answer.eq.O' : answer 컬럼의 값이 'O'

- 비교 연산자 : eq(=), neq(!=), gt(>), gte(>=), lt(<), lte(<=)

In [ ]:
# 3-4. 특정 값 필터 (복수 : OR)

# 조건 정의
condition = 'category.eq.Basic, answer.eq.O'

# .or_()
third_or = select_table_column.or_(condition)

df = pd.DataFrame(third_or.execute().data)

df.head(2)

## **정렬 : 오름차순 / 내림차순**
: 랭킹 or 게시판 시스템에 사용하기 유용

In [ ]:
response = supabase.from_('quiz').select('*').order('sub_category', desc = True).limit(2).execute().data

df = pd.DataFrame(response)
df.head(2)

# **행 추가**

In [ ]:
# 행 정보 입력 -> 컬럼명: 내용
insert_row = {
    'data_code': 'financial_현금흐름',
    'type': 'financial',
    'title': '현금흐름',
    'markdown_text': '현금흐름은 테스트로 넣은 임시 정보입니다.'
}

response = supabase.table('education_data').insert(insert_row).execute()

validation(response)

# **행 삭제**

In [ ]:
# 특정 행 삭제
response = supabase.table('education_data').delete().eq('data_code', 'financial_현금흐름').execute()

validation(response)

# 여러 행 삭제
# delete_primary_keys = ['financial_현금흐름', 'financial_임시데이터']

# supabase.table('education_data').delete().eq('data_code', delete_primary_keys).execute()

# **행 안의 데이터 수정**
- table(): 수정할 테이블
- update(): 업데이트할 내용 -> {컬럼명: 내용}
- eq() : update가 적용될 대상 -> Key, Value

In [ ]:
# 데이터 수정 (테스트)
update_value = {
    'title': 'Test update'  # title 컬럼을 test update로 변환
}

response = supabase.table('education_data').update(update_value).eq('data_code', 'financial_재무상태표').execute()

validation(response)

In [ ]:
# 데이터 원복
update_value = {
    'title': '재무상태표'
}

response = supabase.table('education_data').update(update_value).eq('data_code', 'financial_재무상태표').execute()

validation(response)

In [ ]:
user_info = supabase.from_('user_info').select('id', 'pw').execute().data

df = pd.DataFrame(user_info)

df.head()